In [1]:
from operator import itemgetter
from pydantic import BaseModel, Field, model_validator

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableWithMessageHistory
from langchain_core.messages import HumanMessage, trim_messages
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import ChatOpenAI
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_classic import hub
from langchain_community.chat_message_histories import ChatMessageHistory

In [2]:
prompt = ChatPromptTemplate.from_template(
    '주제 {topic}에 대한 짧은 설명을 해주세요.'
)
model = ChatOpenAI(model='gpt-5-nano', temperature=0)

In [3]:
chain = (
    {'topic':RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print(chain.invoke('더블딥'))

혹시 '더블딥'이 어떤 맥락의 용어인지 조금 더 구체적으로 알려주실 수 있나요? 여러 가지 뜻이 있을 수 있어서요. 가장 흔한 해석 중 하나를 짧게 먼저 설명드리겠습니다.

- 더블 디큐 네트워크(DDQN, Double Deep Q-Network): 강화학습의 한 기법으로, Q-learning의 과대추정 문제를 줄이기 위해 두 개의 신경망을 사용하는 방식입니다.
  - 아이디어: 행동 선택은 온라인 네트워크가 하고, 그 행동의 가치는 타깃 네트워크가 평가합니다. 두 네트워크를 분리해 과대평가를 완화하고 학습의 안정성을 높입니다.
  - 구성: 온라인 네트워크와 일정 시점마다 업데이트되는 타깃 네트워크로 구성.
  - 효과: 수렴과 일반화가 개선되고 학습이 더 안정적으로 진행됩니다.

다른 뜻일 경우 맥락(예: 영화 제목, 주제 이름 등)을 알려주시면 간단히 맞춤 설명해 드리겠습니다.


In [4]:
print(await chain.abatch([{'topic':'더블딥'}, {'topic':'인플레이션'}]))

['더블딥은 한 차례 경기 침체가 끝나고 회복 국면으로 보일 때 다시 경제가 침체로 떨어지는 현상을 말합니다. 보통 수요 부진과 실업 증가, 정책 효과의 소멸 등이 겹쳐 GDP가 다시 마이너스로 돌아가면서 두 차례의 침체가 이어지는 것이 특징입니다. 현대 경제에서 비교적 드물지만, W자형 경기 흐름의 한 예로 설명됩니다.', '인플레이션은 일정 기간 물가가 지속적으로 상승해 화폐의 구매력이 떨어지는 현상입니다. 주된 원인은 총수요 증가, 생산비 증가, 통화 공급 확대 등이 있으며, 보통 CPI로 측정합니다. 중앙은행의 목표는 보통 연간 약 2%의 물가상승률을 유지하는 것입니다. 인플레이션이 높아지면 가계의 실질구매력이 감소하고, 대출 이자 부담과 저축의 가치에 영향이 나타납니다.']


In [5]:
for token in chain.stream({'topic':'더블딥'}):
    print(token, end='', flush=True)

더블딥은 경제가 한 차례 회복 국면을 보이다가 다시 침체로 떨어지는 현상을 말합니다. 즉, 잠깐의 반등 뒤에 GDP가 다시 감소하고 실업이 악화되는 상황이 반복되는 것을 뜻합니다. 주된 원인으로는 소비와 투자 부진, 과다한 부채, 정책의 효과 시차, 외부 충격 등이 작용합니다.

In [6]:
eng_prompt = ChatPromptTemplate.from_template('이 대답을 영어로 번역해주세요.: {answer}')

In [7]:
composed_chain = (
    {'answer':chain}
    | eng_prompt
    | model
    | StrOutputParser()
)

for token in composed_chain.stream({'topic':'인플레이션'}):
    print(token, end='', flush=True)

Here is a brief translation:

- Definition: The phenomenon of the general price level rising steadily over a period of time, causing the purchasing power of money to fall.
- Measurement: Usually expressed as the annual inflation rate, represented by the CPI (Consumer Price Index).
- Causes: Demand-pull inflation where demand outpaces supply, cost-push inflation from rising costs, and expectations of inflation, among others.
- Effects: Increased cost of living, a decrease in the real value of savings (or a relief in debtors' burden), and possible changes in asset prices.
- Policy: Central banks attempt to manage inflation through interest-rate policies and by setting price-stability targets.

Simple example: If prices rise by 3% per year, the amount of money needed to buy the same goods five years later will be about 1.16 times the current amount.

In [8]:
kor_chain = (
    ChatPromptTemplate.from_template('{topic}에 대해 짧은 설명을 해주세요.')
    | model
    | StrOutputParser()
)

eng_chain = (
    ChatPromptTemplate.from_template('{topic}에 대해 짧게 영어로 설명을 해주세요.')
    | model
    | StrOutputParser()
)

parallel_chain = RunnableParallel(kor=kor_chain, eng=eng_chain)
result = await parallel_chain.ainvoke({'topic':'인플레이션'})

print(result['kor'])
print('--'*30)
print(result['eng'])

인플레이션은 일정 기간 동안 재화와 서비스의 일반 물가가 지속적으로 오르는 현상으로, 돈의 구매력이 떨어지는 것을 말합니다.  
주된 원인으로는 수요가 공급보다 빨리 늘어나는 경우(수요 견인), 생산비 상승으로 가격이 오르는 경우(비용 상승), 그리고 기대인플레이션이 형성될 때의 현상 등이 있습니다.  
물가 변화는 보통 CPI 같은 물가지수로 측정되며, 중앙은행은 물가를 안정시키려 금리 정책 등을 활용합니다.
------------------------------------------------------------
Inflation is the rate at which the general price level of goods and services rises over time, meaning your money buys less. It can happen when demand is high, costs rise, or the money supply grows faster than the economy. It’s usually measured by price indices like the Consumer Price Index (CPI).


In [9]:
template = PromptTemplate.from_template('주제 {topic}에 대해 금융 관련 짧은 조언을 해주세요.')
print(template.invoke({'topic':'투자'}))
print(template.invoke('투자'))

text='주제 투자에 대해 금융 관련 짧은 조언을 해주세요.'
text='주제 투자에 대해 금융 관련 짧은 조언을 해주세요.'


In [10]:
chain = (
    template
    | model
    | StrOutputParser()
)
print(chain.invoke({'topic':'더블딥'}))

더블딥은 단기간에 경기 침체가 두 차례 반복될 가능성을 말합니다. 이에 대비하는 짧은 조언:

- 현금 여력 확보: 비상금 3–6개월 생활비를 유지해 변동기에 버티기.
- 부채 관리: 고금리 빚은 먼저 상환하고 신용카드 잔액은 줄이기.
- 방어적 포트폴리오 우선: 필수소비재, 헬스케어, 유틸리티 같은 방어주와 고질적 현금흐름이 좋은 자산 비중 확대.
- 채권 포지션 점검: 단기채/인플레이션 연동채(TIPS)로 금리 상승 리스크를 분산.
- 분할 매수(DCA): 하락 시점에 한 번에 매수하기보다 주기적으로 매수해 가격 변동 리스크 낮추기.
- 비용 최소화: 저비용 인덱스/패시브 펀드 선호로 수수료 부담 줄이기.
- 리밸런싱 원칙 준수: 목표 자산 비율을 주기적으로 맞춰 과도한 편향 방지.
- 장기 목표 유지: 단기 변동에 흔들리지 말고 투자 철학과 목표를 재확인.

필요하면 귀하의 상황(목표, 기간, 현재 포트폴리오)을 바탕으로 구체적인 조정안을 제안해 드리겠습니다.


In [11]:
template = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다.'),
     ('user', '주제 {topic}에 대해 금융 관련 주언을 해주세요.')]
)
print(template.invoke({'topic':'주식'}))
print(template.invoke('주식'))

messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='주제 주식에 대해 금융 관련 주언을 해주세요.', additional_kwargs={}, response_metadata={})]
messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='주제 주식에 대해 금융 관련 주언을 해주세요.', additional_kwargs={}, response_metadata={})]


In [12]:
template = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다.'),
     ('placeholder', '{msgs}')]
)
print(template.invoke({'msgs':[HumanMessage(content='안녕하세요.')]}))

messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요.', additional_kwargs={}, response_metadata={})]


In [13]:
examples = [
    {
        "question": "주식 투자와 예금 중 어느 것이 더 수익률이 높은가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자
""",
    } ,
    {
        "question": "부동산과 채권 중 어느 것이 더 안정적인 투자처인가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권
""",
    },
]

In [14]:
embeddings = HuggingFaceEmbeddings(model='BAAI/bge-m3', model_kwargs={'device':'cuda'}, encode_kwargs={'batch_size':8})

In [15]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=embeddings,
    vectorstore_cls=Chroma,
    k=1
)

question = '부동산 투자의 장점은 무엇인가?'
selected_examples = example_selector.select_examples({'question':question})

for example in selected_examples:
    for k, v in reversed(example.items()):
        print(f'{k}: {v}')

question: 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?
answer: 
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권



In [16]:
example_prompt = ChatPromptTemplate.from_messages(
    [('human', '{question}'),
     ('ai', '{answer}')]
)

fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt
)

prompt = ChatPromptTemplate.from_messages(
    [('system', '당신은 금융 전문가입니다.'),
     fewshot_prompt,
     ('human', '{question}')]
)

In [17]:
chain = prompt | model
response = chain.invoke({'question':'부동산 투자의 장점은 무엇인가?'})
print(response.content)

부동산 투자에서 흔히 꼽히는 장점은 다음과 같습니다.

- 안정적인 현금 흐름: 임대료를 통한 정기적 현금 수입이 가능해 현금흐름이 비교적 안정적일 수 있습니다.
- 자본 상승 가능성: 시간이 흐르면서 토지나 건물 가치가 상승해 자본이익이 생길 수 있습니다.
- 레버리지 활용: 대출을 활용해 초기 자본 대비 투자 규모를 키워 수익률을 높일 수 있습니다.
- 포트폴리오 다변화 및 위험 분산: 주식과 다른 움직임을 보이는 경우가 있어 포트폴리오의 균형과 리스크 분산에 도움이 됩니다.
- 인플레이션 헤지 효과: 물가 상승 시 임대료와 자산가치가 함께 상승하는 경우가 있어 인플레이션에 대한 방어 수단이 됩니다.
- 실물자산의 안정감 및 관리 가능성: 물리적 자산이므로 관리 전략에 따라 가치를 직접 끌어올릴 수 있습니다.
- 가치 창출 기회: 리노베이션, 운영 효율화, 임대료 재설정 등으로 추가 수익 및 가치 상승이 가능합니다.
- 세제 혜택 및 비용 처리 가능성: 지역별 세법에 따라 이자 비용 공제, 감가상각 등으로 세무 혜택을 얻을 수 있습니다.

참고로 장점은 지역·시장 상황 및 투자 유형(주거, 상업용, 토지 등)에 따라 달라지므로, 본인의 목표와 리스크 허용도에 맞춰 신중히 평가하는 것이 좋습니다.


In [18]:
prompt = hub.pull('hardkothari/prompt-maker')
print(prompt)

input_variables=['lazy_prompt', 'task'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hardkothari', 'lc_hub_repo': 'prompt-maker', 'lc_hub_commit_hash': 'c5db8eeefa7be4862a9599b759608dd10ee53f53910838f69abb5ab31c257c2d'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert Prompt Writer for Large Language Models.\n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['lazy_prompt', 'task'], input_types={}, partial_variables={}, template='Your goal is to improve the prompt given below for {task} :\n--------------------\n\nPrompt: {lazy_prompt}\n\n--------------------\n\nHere are several tips on writing great prompts:\n\n-------\n\nStart the prompt by stating that it is an expert in the subject.\n\nPut instructions at the beginning of the prompt and use ### or to separate the instruction and context \n\nBe specific, descriptive and as deta

In [19]:
class FinancialAdvice(BaseModel):
    setup: str = Field(description='금융 조언 상황을 설정하기 위한 질문')
    advice: str = Field(description='질문을 해결하기 위한 금융 답변')

    @model_validator(mode='before')
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get('setup', '')
        if not setup.endswith('?'):
            raise ValueError('질문은 ?로 끝나야 합니다.')
        
        return values

In [20]:
parser = PydanticOutputParser(pydantic_object=FinancialAdvice)

prompt = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다. \n반드시 아래의 포맷 규칙을 준수하여 답변하세요. \n{format_instructions}'),
     ('human', '{query}')]
)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser

In [21]:
try:
    result = chain.invoke({'query':'부동산에 관련하여 금융 조언을 받을 수 있게 질문하세요. 질문은 ?로 끝나야 합니다.'})
    print(result)
except Exception as e:
    print(f'오류 발생: {e}')

setup='부동산 투자용 대출과 현금 흐름 관리에 초점을 맞춰, 1채의 투자용 부동산을 구입하려는 상황에서 어떤 금융 전략으로 자금을 조달하고 리스크를 관리하는 것이 가장 효과적입니까?' advice='다음 프레임으로 자금 조달과 현금 흐름을 점검하는 것이 유리합니다:\n1) 예산과 목표: 투자 총액, 예상 수익률 목표, 다운페이먼트 20% 이상, 초기 비용 여유금 확보\n2) 자금 조달 옵션: 고정금리 vs 변동금리, 대출 기간(15/20/30년), LTV 70-75% 범위, 금리 시나리오에 따른 DSCR 목표 1.25 이상 유지\n3) 현금 흐름 분석: 임대료 예측, 공실률 반영, 관리비/보험/세금/수리비 포함, 월 순현금흐름 산정; 대출 원리금 상환 후 남는 금액 확인\n4) 리스크 관리: 금리 상승과 공실 리스크에 대비한 예비비 3-6개월 원리금 확보, 다각화나 보수적 가정 적용\n5) 세무/비용: 취득세, 재산세, 양도소득세, 감가상각 등 세무 영향 사전 파악; 세무사 상담 권장\n6) 실행 로드맵: 사전 승인 받기, 매물 분석, 제안 및 협상, 계약 및 클로징 순으로 진행\n7) 지역 특성 반영: 지역 임대 수요와 시세 경향, 개발 계획, 규제 변화 모니터링\n8) 최종 권고: 여러 시나리오로 모의 실행해 의사결정을 내리고 필요 시 부동산 투자 컨설턴트, 대출 담당자, 세무사 등 전문가와 상담'


In [22]:
prompt = ChatPromptTemplate.from_messages(
    [('system', '당신은 금융 상담사입니다. 사용자에게 최선의 금융 조언을 제공합니다.'),
     ('placeholder', '{msgs}')]
)

chain = prompt | model

In [23]:
chat_history = ChatMessageHistory()

chat_history.add_user_message('저축을 늘리기 위해서 무엇을 할 수 있나요?')
chat_history.add_ai_message('저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요.')
chat_history.add_user_message('방금 뭐라고 했나요?')

ai_response = chain.invoke({'msgs':chat_history.messages})
print(ai_response.content)

방금 제 답변은 “저축 목표를 설정하고 매달 자동 이체로 일정 금액을 저축하라”였습니다.

원하시면 더 구체적으로 실행 방법도 드릴게요. 예를 들어:
- 목표 설정: 목표 금액, 달성 기간, 긴급자금 여부를 정하기
- 예산 파악: 월 지출 분석 후 저축 가능 금액 산정
- 자동 이체: 급여일이나 매월 같은 날에 저축계좌로 자동 이체 설정
- 비상금 먼저: 최소 3~6개월 생활비를 비상금으로 확보
- 정기 점검: 매월 말 진행 상황 점검 및 필요 시 금액 조정

필요한 경우, 귀하의 소득/지출 상황에 맞춘 맞춤 계획도 만들어 드릴게요. 알려주실 수 있을까요?


In [24]:
chat_history = ChatMessageHistory()

prompt = ChatPromptTemplate.from_messages(
    [('system', '당신은 금융 상담사입니다. 모든 질문에 최선을 다해 답변하십시오. 일단은 간략하게 설명해준 후, 같은 내용에 대해서 추가적인 설명을 요청하면 더 자세히 답변하십시오.'),
     ('placeholder', '{chat_history}'),
     ('user', '{question}')]
)

chain = prompt | model

In [25]:
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key='question',
    history_messages_key='chat_history'
)

In [26]:
result = chain_with_message_history.invoke(
    {'question':'저축을 늘리기 위해 무엇을 할 수 있나요?'},
    {'configurable':{'session_id':'unused'}}
)
print(result.content)

간단 요약부터 시작하면:

- 핵심은 지출 관리와 자동저축을 통해 "남는 돈을 꾸준히 저축으로 옮겨두는 것"입니다. 여기에 긴급자금 마련, 부채 관리, 필요하면 수입 늘리기도 함께하면 효과가 큽니다.

좀 더 구체적인 실행 방법 (빠르게 적용 가능)
- 현재 지출 파악: 한 달간 모든 지출을 기록해 필요 vs 불필요를 구분합니다.
- 예산 짜기: 고정비와 필수비를 먼저 확보하고, 비필수 소비를 줄이기 위한 규칙을 세웁니다(예: 50/30/20 또는 본인 상황에 맞게 조정).
- 자동저축 설정: 월급일에 일정 금액을 별도 저축계좌로 자동 이체되도록 설정합니다.
- 긴급자금 마련: 최소 3~6개월의 생활비를 커버하는 자금을 먼저 마련합니다.
- 지출 절감 전략: 구독료 재검토, 보험료 비교, 에너지 절약 등 반복적 비용 줄이기.
- 부채 관리: 금리가 높은 부채부터 우선 상환하거나 재융자를 검토합니다.
- 소득 증가 아이디어: 부업 시작이나 기술/자격증 취득으로 수입을 늘릴 방법을 모색합니다.
- 저장 수단 다변화: 단기는 예금/적금 위주, 장기적으로는 위험 허용 범위에 맞춘 투자도 고려해보기(장기 목표라면 투자 병행이 일반적).
- 세제 혜택/금융상품 활용: 지역에 따라 세액공제나 혜택이 있는 저축상품 활용을 확인합니다.

추가로 원하시면, 당신의 상황에 맞춘 더 자세한 로드맵(목표 금액/기간, 월 저축액 목표, 구체적 계좌 선정 등)을 만들어 드리겠습니다. 아래 정보를 알려주시면 개인화된 계획으로 정리해 드릴게요.
- 월간 가처분 소득(세전/세후 중 어떤지)
- 현재 저축액 및 부채 현황(금액, 금리)
- 목표 저축액/목표 기간
- 거주 지역(나라) 및 이용 가능한 저축/투자 계좌 유형
- 위험 선호도(보수적/중립적/공격적)와 투자 의향 여부

필요하신 만큼 간략한 개요에서 더 자세한 계획으로 단계별로 설명해 드리겠습니다.


In [27]:
result = chain_with_message_history.invoke(
    {'question':'알려준 전략을 바탕으로 저축을 늘려서 집을 살 수 있을까요?'},
    {'configurable':{'session_id':'unused'}}
)
print(result.content)

가능합니다. 다만 목표 주택 가격, 필요한 다운페이먼트 비율(지역에 따라 다름), 구입 시점의 대출 조건 등에 따라 달라집니다. 기본 아이디어는 앞서 제시한 “저축 자동화 + 지출 관리 + 긴급자금 확보” 전략을 주택 구입 자금 마련에 맞춰 구체화하는 것입니다.

간단한 실행 로드맵(바로 적용 가능한 순서)
- 목표 설정: 희망하는 주택 가격대와 필요한 다운페이먼트 금액(예: 20% 가정 시 필요한 금액) 결정.
- 기간 설정: 언제까지 자금을 모을지 목표 기간 정하기.
- 월 저축 목표 산출: 남은 기간 동안 필요한 다운페이먼트를 현재 저축으로 뺀 뒤 월 단위로 나눠 목표 금액 산출.
- 자동저축 강화: 월급일 또는 지정일에 자동 이체로 별도 계좌에 축적.
- 지출 관리: 필수/비필수 구분, 불필요 지출 줄이기, 구독료·보험료 재검토.
- 긴급자금 확보: 먼저 3~6개월 생활비 수준의 비상금은 따로 유지.
- 부채 관리: 금리가 높은 부채가 있다면 우선 상환 또는 재융자 검토.
- 소득 증가 고려: 부업, 업스킬, 자격증 취득 등을 통해 수입 늘리기.
- 대출 준비: 신용점수 관리, 지역의 주택담보대출 상품 비교, 사전승인(Pre-Approval) 절차 준비.

원하시면, 당신의 상황에 맞춘 구체적 로드맵을 만들어 드리겠습니다. 아래 정보를 알려주시면 개인화된 계획으로 정리해 드립니다.
- 예상 월간 순소득(또는 세후 금액)
- 현재 예금/저축액과 부채(금액, 금리)
- 목표 주택 가격과 원하시는 다운페이먼트 비율(예: 20%)
- 원하는 구매 시점(연도)
- 거주 지역(국가/도시) 및 이용 가능한 금융상품 유형
- 위험 허용도와 투자 의향(예: 보수적/중립적/공격적)

필요하신 만큼 간략한 개요에서 시작해, 원하시면 바로 맞춤형 로드맵으로 자세히 설명드리겠습니다.


In [28]:
trimmer = trim_messages(strategy='last', max_tokens=3, token_counter=len)

In [29]:
chain_with_trimming = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)
    | prompt 
    | model
)

In [30]:
chain_with_trimmed_history = RunnableWithMessageHistory(
    chain_with_trimming,
    lambda session_id: chat_history,
    input_messages_key='question',
    history_messages_key='chat_history'
)

In [31]:
result = chain_with_trimmed_history.invoke(
    {'question':'언급해준 내용을 세 문장으로 간략하게 정리해줄 수 있나요?'},
    {'configurable':{'session_id':'unused'}}
)
print(result.content)

집을 살 자금을 모으려면 지출 관리와 자동저축으로 매달 일정 금액을 주택자금 계좌로 옮기는 전략이 핵심입니다. 다운페이먼트 목표(예: 가격의 20%), 기간 설정, 그리고 월 저축액 계산을 통해 구체적인 로드맵을 만들고 긴급자금 3~6개월 분은 별도 비상계좌로 유지하며 필요 없는 지출을 줄입니다. 또한 신용 관리, 부채 재조정/재융자 검토, 소득 증가 아이디어, 대출 사전승인 준비 등 대출 관련 요소를 함께 준비해 실제 구매 시점에 대비합니다.


In [32]:
def summarize_messages(chain_input):
    stored_messages = chat_history.messages

    if len(stored_messages) == 0:
        return False
    
    summarization_prompt = ChatPromptTemplate.from_messages(
        [('placeholder', '{chat_history}'),
         ('user', '이전 대화를 요약해 주세요. 가능한 많은 세부 정보를 포함하십시오.')]
    )

    summarization_chain = summarization_prompt | model
    summary_message = summarization_chain.invoke({'chat_history':stored_messages})

    chat_history.clear()
    chat_history.add_message(summary_message)

    return True

In [33]:
chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

In [34]:
result = chain_with_summarization.invoke(
    {'question':'저에게 어떤 조언을 해주셨나요?'},
    {'configurable':{'session_id':'unused'}}
)

print(result.content)

요약해 드리면, 주택 구입 자금을 모으기 위해 다음과 같은 조언을 드렸습니다.

- 목표 설정과 자동저축: 주택다운페이먼트 목표를 정하고(예: 가격의 20%), 기간에 맞춰 월 저축액을 계산한 뒤 자동이체로 주택자금 계좌에 저축합니다.
- 지출 관리와 긴급자금: 필요와 불필요 지출을 구분하고 구독료 등 불필요 지출을 줄이며, 3~6개월 생활비 규모의 긴급자금을 별도 비상계좌에 확보합니다.
- 부채 관리: 금리가 높은 부채를 먼저 상환하거나 재융자를 검토해 이자 부담을 줄입니다.
- 소득 증가와 자금 운용 다각화: 부업이나 업스킬링으로 소득을 늘리고, 자금 운용은 단기 예금/적금 위주에서 시작해 필요 시 장기 투자도 검토합니다.
- 대출 준비: 신용점수 관리, 지역별 주택담보대출 상품 비교, 사전승인(Pre-Approval) 절차를 준비합니다.
- 맞춤 계획을 위한 정보 요청: 현재 수입, 예금/부채 규모와 금리, 목표 주택 가격/다운페이먼트 비율, 구매 시점, 거주 지역 및 위험 선호도 등 구체적인 수치를 요청드립니다.
- 맞춤 로드맵 제공은 수치를 주시면 바로 시작: 귀하의 상황에 맞춘 구체적 로드맵이나 실행 체크리스트를 만들어 드리겠습니다.

원하시면 이 내용을 바탕으로 아주 간단한 실행 체크리스트나 초안 로드맵을 바로 만들어 드릴게요. 현재 가지고 계신 수치를 몇 가지 알려주실 수 있을까요? 예: 예상 월순소득, 현재 저축액/부채 규모, 목표 주택가격, 목표 다운페이먼트 비율, 구매 희망 시점.
